# **Predicting Media Memorability**
# This notebook is organized as follows:


1. **Definitions the Functions**

2. **Loading the Functions**

3. **Data Preprocessing for Captions**

4. **Applying Different Machine Learning Models**

5. **Selecting the best model for final model**

6. **Training Full Dev-Set**

7. **Predicting the Final Result** 











In [190]:
pip install pyprind

In [0]:
import pandas as pd
import numpy as np
from string import punctuation
import pyprind
from collections import Counter
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
import glob
import nltk

# **1. Defining the Functions**

In [0]:
#Function to load C3D features
def load_c3d(captions, c3dPath):
    files = list(captions["video"].values)
    c3dfeatures = []
    for file in files:
        file = c3dPath+file[:-4]+'txt'
        c3dfeatures.append(np.loadtxt(file))
    #print(type(c3dfeatures))
    return c3dfeatures

In [0]:
#Function to load the captions into dataframes
def load_captions(filename):
    video_name = []
    captions = []
    dataframe = pd.DataFrame()
    with open(filename) as file:
        for line in file:
            pair = line.split() #each line in the text file contains to words so, this code will split them into two words
            video_name.append(pair[0]) #first word will be assigned as video name
            captions.append(pair[1]) #second word will be assigned as caption
        dataframe['video']=video_name #setting these two as column names of dataframe
        dataframe['caption']=captions
    return dataframe

In [0]:
#Function to calculate Spearman coefficient scores
def Get_score(Y_pred,Y_true):
    '''Calculate the Spearmann"s correlation coefficient'''
    Y_pred = np.squeeze(Y_pred)
    Y_true = np.squeeze(Y_true)
    if Y_pred.shape != Y_true.shape:
        print('Input shapes don\'t match!')
    else:
        if len(Y_pred.shape) == 1:
            Res = pd.DataFrame({'Y_true':Y_true,'Y_pred':Y_pred})
            score_mat = Res[['Y_true','Y_pred']].corr(method='spearman',min_periods=1)
            print('The Spearman\'s correlation coefficient is: %.3f' % score_mat.iloc[1][0])
        else:
            for ii in range(Y_pred.shape[1]):
                Get_score(Y_pred[:,ii],Y_true[:,ii])

## **2. Loading Feature**

1.   **Loading C3D Features.**
2.   **Loading Captions.**
3.   **Loading Ground Truth.**
4.   **Loading Asthetics Features.**
 








In [195]:
from google.colab import drive
import os
drive.mount('/content/gdrive/')
os.chdir('/content/gdrive/My Drive/CA684_Assignment/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [0]:
#load the ground truth dataset
csv_path ='./Dev-set/Ground-truth/ground-truth.csv'
dataset = pd.read_csv(csv_path)

#load captions
captions_path ='./Dev-set/Captions/dev-set_video-captions.txt'
captions = load_captions(captions_path)


In [0]:
#load C3D features
c3dPath = './Dev-set/C3D/'
c3dfeatures = load_c3d(captions,c3dPath)

In [198]:
captions.head()

,video,caption
0,video3.webm,blonde-woman-is-massaged-tilt-down
1,video4.webm,roulette-table-spinning-with-ball-in-closeup-shot
2,video6.webm,khr-gangsters
3,video8.webm,medical-helicopter-hovers-at-airport
4,video10.webm,couple-relaxing-on-picnic-crane-shot


In [199]:
dataset.head()

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations
0,video3.webm,0.924,34,0.846,13
1,video4.webm,0.923,33,0.667,12
2,video6.webm,0.863,33,0.700,10
3,video8.webm,0.922,33,0.818,11
4,video10.webm,0.950,34,0.900,10


**Loading Asthetic Features**


In [0]:
#function to read aesthetic feature
def aes_fea_read(df, filename):
 df1 = pd.read_csv(filename, header=None)
 df1['video'] = filename[filename.find('video'):filename.find('.txt')] + '.webm'
 df = df.append(df1, ignore_index=True)
 return df

In [0]:
def reading_file_names(path):
 files = []
 file_number=0
 # r=root, d=directories, f = files
 for r, d, f in os.walk(path):
     for file in f:
         if '.txt' in file:
             files.append(os.path.join(r, file))
             file_number = file_number + 1
 print("total files read = " + str(file_number))
 return files

In [202]:

#read aesthetic feature train data
path='/content/gdrive/My Drive/aesthetic_visual_features/aesthetic_feat_dev-set_mean' #(need to chnage the path as per the directory the C3D or any other files are stored in)
files = reading_file_names(path)
dataframe_aes = pd.DataFrame()
for filename in files:
   dataframe_aes = aes_fea_read(dataframe_aes, filename)

total files read = 6000


In [203]:
#shape of the dataframes
print(f'Ground Truth : {dataset.shape}')
print(f'Dev Captions : {captions.shape}')
print(f'C3D          : ({len(c3dfeatures)})')
print(f'Asthetic features : ({ len(dataframe_aes)})')


Ground Truth : (6000, 5)
Dev Captions : (6000, 2)
C3D          : (6000)
Asthetic features : (6000)


## **3. Data Processing for Captions**

In [204]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [205]:
#loading the nltk stopwords of English
stopwords = nltk.corpus.stopwords.words('english')
print(f'Length of Stopwords: {len(stopwords)}')

Length of Stopwords: 179


In [206]:
#Removing punctuations and stop words from captions
# setup prograss tracker
pbar = pyprind.ProgBar(len(captions['caption']), title='Counting word occurrences')
for i, cap in enumerate(captions['caption']):
    # replace punctuations with space
    # convert words to lower case 
    text = ''.join([c if c not in punctuation else ' ' for c in cap]).lower()
    #removing stopwords
    rmv_stopwords= ' '.join([word for word in text.split() if word not in stopwords])
    captions.loc[i,'caption'] = rmv_stopwords #updating the original captions 
    pbar.update()

Counting word occurrences
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


In [207]:
captions.head()

,video,caption
0,video3.webm,blonde woman massaged tilt
1,video4.webm,roulette table spinning ball closeup shot
2,video6.webm,khr gangsters
3,video8.webm,medical helicopter hovers airport
4,video10.webm,couple relaxing picnic crane shot


In [208]:
#implementing bag of words for the combined captions
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word",max_features=3112) 
captions_bag = vectorizer.fit_transform(captions.caption).toarray()
type(captions_bag)

numpy.ndarray

In [209]:
captions_bag.shape

(6000, 3112)

In [0]:
captions_c3d = (captions_bag.tolist())
counter = 0
for item in range(6000):
    captions_c3d[counter] = np.append(captions_c3d[counter],c3dfeatures[counter],axis=0)
    counter = counter+1

In [211]:
len(captions_c3d[0])

3213

In [212]:
len(captions_c3d)

6000

# **4. Applying Different Machine Learning Models**




**Model for Captions- Extra trees Regressor**




In [0]:
X = captions_bag
y = dataset[['short-term_memorability','long-term_memorability']].values

In [0]:
# Splitting the dataset into the Training set and Test set
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)


In [215]:
print('X_train ', X_train.shape)
print('X_test  ', X_test.shape)
print('Y_train ', y_train.shape)
print('Y_test  ', y_test.shape)

X_train  (4800, 3112)
X_test   (1200, 3112)
Y_train  (4800, 2)
Y_test   (1200, 2)


In [0]:
from sklearn.ensemble import ExtraTreesRegressor
captions_rf = ExtraTreesRegressor (n_estimators = 10, n_jobs = -1, random_state= 45)

In [0]:
captions_rf.fit(X_train,y_train)

ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=10, n_jobs=-1, oob_score=False,
                    random_state=45, verbose=0, warm_start=False)

In [0]:
captions_pred = captions_rf.predict(X_test)

In [219]:
Get_score(captions_pred, y_test)

The Spearman's correlation coefficient is: 0.349
The Spearman's correlation coefficient is: 0.115


**Model for Captions- Multi-Output Regressor**

In [0]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor

In [221]:
cap_multirf = MultiOutputRegressor(RandomForestRegressor( random_state=45))
cap_multirf.fit(X_train, y_train)

MultiOutputRegressor(estimator=RandomForestRegressor(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     criterion='mse',
                                                     max_depth=None,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     max_samples=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
                                                     n_estimators=100,
                                              

In [0]:
cap_pred = cap_multirf.predict(X_test)

In [223]:
Get_score(cap_pred, y_test)

The Spearman's correlation coefficient is: 0.418
The Spearman's correlation coefficient is: 0.177


**Model for Caption - 'K-Nearest Neighbors'**

In [0]:
X = captions_bag
y = dataset[['short-term_memorability','long-term_memorability']].values

# Splitting the dataset into the Training set and Test set
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor(n_neighbors=300)

In [225]:
model.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=300, p=2,
                    weights='uniform')

In [0]:
KNN_pred = model.predict(X_test)

In [227]:
Get_score(KNN_pred, y_test)

The Spearman's correlation coefficient is: 0.374
The Spearman's correlation coefficient is: 0.127


**Model for Captions - GradientBoostingRegressor**

In [0]:
# Separate the training and test data into arrays of a single target value
y_train_0 = y_train[:,0]
y_train_1 = y_train[:,1]

y_test_0 = y_test[:,0]
y_test_1 = y_test[:,1]

# Import the model
from sklearn.ensemble import GradientBoostingRegressor

# Train the individual models with the default hyperparameters
gbr0 = GradientBoostingRegressor()
gbr0.fit(X_train,y_train_0)

gbr1 = GradientBoostingRegressor()
gbr1.fit(X_train,y_train_1)

y_gbr0 = gbr0.predict(X_test)
y_gbr1 = gbr1.predict(X_test)

# Combine the predicted values so that we get an array of (150L, 2L)
y_gbr = np.concatenate((y_gbr0.reshape(-1,1),y_gbr1.reshape(-1,1)),axis=1)

In [229]:
Get_score(y_gbr, y_test)

The Spearman's correlation coefficient is: 0.400
The Spearman's correlation coefficient is: 0.130


**Model for C3D features - Extra Trees Regressor**

In [0]:
X = c3dfeatures
y = dataset[['short-term_memorability','long-term_memorability']].values

In [0]:
# Splitting the dataset into the Training set and Test set
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [0]:
from sklearn.ensemble import ExtraTreesRegressor
rf_regressor = ExtraTreesRegressor (n_estimators = 10, n_jobs = -1, random_state= 45)

In [233]:
rf_regressor.fit(X_train,y_train)

ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=10, n_jobs=-1, oob_score=False,
                    random_state=45, verbose=0, warm_start=False)

In [0]:
rf_pred = rf_regressor.predict(X_test)

In [235]:
Get_score(rf_pred, y_test)

The Spearman's correlation coefficient is: 0.254
The Spearman's correlation coefficient is: 0.079


**Model for C3D features- Multi-Output Regressor**

In [0]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor

In [237]:
regr_multirf = MultiOutputRegressor(RandomForestRegressor( random_state=45))
regr_multirf.fit(X_train, y_train)

MultiOutputRegressor(estimator=RandomForestRegressor(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     criterion='mse',
                                                     max_depth=None,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     max_samples=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
                                                     n_estimators=100,
                                              

In [0]:
m_pred = regr_multirf.predict(X_test)

In [239]:
Get_score(m_pred, y_test)

The Spearman's correlation coefficient is: 0.330
The Spearman's correlation coefficient is: 0.114


**Model for C3D features - 'K-Nearest Neighbors'**

In [0]:
from sklearn.neighbors import KNeighborsRegressor
model_C3D = KNeighborsRegressor(n_neighbors=150)

In [241]:
model_C3D.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=150, p=2,
                    weights='uniform')

In [0]:
KNN_C3D_pred = model_C3D.predict(X_test)

In [243]:
Get_score(KNN_C3D_pred, y_test)

The Spearman's correlation coefficient is: 0.258
The Spearman's correlation coefficient is: 0.112


**Model for C3D features - Gradient Boosting Regressor**

In [0]:
X = c3dfeatures
y = dataset[['short-term_memorability','long-term_memorability']].values

# Splitting the dataset into the Training set and Test set
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)


# Separate the training and test data into arrays of a single target value
y_train_0 = y_train[:,0]
y_train_1 = y_train[:,1]

y_test_0 = y_test[:,0]
y_test_1 = y_test[:,1]

# Import the model
from sklearn.ensemble import GradientBoostingRegressor

# Train the individual models with the default hyperparameters
gbr_c3d_1 = GradientBoostingRegressor(n_estimators = 200,  random_state= 10)
gbr_c3d_1.fit(X_train,y_train_0)

gbr_c3d_2 = GradientBoostingRegressor(n_estimators = 200,  random_state= 10)
gbr_c3d_2.fit(X_train,y_train_1)

y_gbr_c3d_1 = gbr_c3d_1.predict(X_test)
y_gbr_c3d_2 = gbr_c3d_1.predict(X_test)

# Combine the predicted values so that we get an array of ( short-term,long-term )
y_gbr_c3d = np.concatenate((y_gbr_c3d_1.reshape(-1,1),y_gbr_c3d_2.reshape(-1,1)),axis=1)

In [245]:
Get_score(y_gbr_c3d, y_test)

The Spearman's correlation coefficient is: 0.313
The Spearman's correlation coefficient is: 0.164


**Model for Asthetic features - Extra Trees Regressor**

In [0]:
x=dataframe_aes
y = dataset[['short-term_memorability','long-term_memorability']].values


In [0]:
# Splitting the dataset into the Training set and Test set
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [0]:
from sklearn.ensemble import ExtraTreesRegressor
rf_asthetics = ExtraTreesRegressor (n_estimators = 10, n_jobs = -1, random_state= 45)

In [249]:
rf_asthetics.fit(X_train,y_train)

ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=10, n_jobs=-1, oob_score=False,
                    random_state=45, verbose=0, warm_start=False)

In [0]:
rf_aes_pred = rf_asthetics.predict(X_test)

In [251]:
Get_score(rf_aes_pred,y_test)

The Spearman's correlation coefficient is: 0.254
The Spearman's correlation coefficient is: 0.079


**Model for Aesthetics Features- Multi-Output Regressor**

In [252]:
aes_multirf = MultiOutputRegressor(RandomForestRegressor( random_state=45))
aes_multirf.fit(X_train, y_train)

MultiOutputRegressor(estimator=RandomForestRegressor(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     criterion='mse',
                                                     max_depth=None,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     max_samples=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
                                                     n_estimators=100,
                                              

In [0]:
aes_pred = aes_multirf.predict (X_test)

In [254]:
Get_score(aes_pred,y_test)

The Spearman's correlation coefficient is: 0.330
The Spearman's correlation coefficient is: 0.114


**Model for Asthetic features - 'K-Nearest Neighbors'**

In [0]:
from sklearn.neighbors import KNeighborsRegressor
model_aes = KNeighborsRegressor(n_neighbors=150)

In [256]:
model_aes.fit(X_train,y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=150, p=2,
                    weights='uniform')

In [0]:
model_aes_pred = model_aes.predict(X_test)

In [258]:
Get_score(model_aes_pred,y_test)

The Spearman's correlation coefficient is: 0.258
The Spearman's correlation coefficient is: 0.112


**Model for Asthetic features - Gradient Boosting Regressor**

In [0]:
# Separate the training and test data into arrays of a single target value
y_train_0 = y_train[:,0]
y_train_1 = y_train[:,1]

y_test_0 = y_test[:,0]
y_test_1 = y_test[:,1]

# Import the model
from sklearn.ensemble import GradientBoostingRegressor

import time
st=time.time()

# Train the individual models with the default hyperparameters
gbr_aes1 = GradientBoostingRegressor()
gbr_aes1.fit(X_train,y_train_0)

gbr_aes2 = GradientBoostingRegressor()
gbr_aes2.fit(X_train,y_train_1)

y_gbr_a1 = gbr_aes1.predict(X_test)
y_gbr_a2 = gbr_aes2.predict(X_test)

# Combine the predicted values so that we get an array of (150L, 2L)
y_gbr_aes = np.concatenate((y_gbr_a1.reshape(-1,1),y_gbr_a2.reshape(-1,1)),axis=1)

In [260]:
Get_score(y_gbr_aes,y_test)

The Spearman's correlation coefficient is: 0.310
The Spearman's correlation coefficient is: 0.111



|Features|Models|Short Term| Long Term |
|------|--------|------|-------------------|
|Captions|Extra-Tree Regressor|0.349|0.115|
||Multi-Output Regressor|**0.418**|**0.177**|
||K-Nearest Neighbors Regressor|0.314|0.127|
||Gradient Boosting Regressor|0.400|0.130|
|C3D|Extra-Tree Regressor|0.254|0.079|
||Multi-Output Regressor|0.330|0.114|
||K-Nearest Neighbors Regressor|0.258|0.112|
||Gradient Boosting Regressor|0.313|0.164|
|Aesthetics|Extra-Tree Regressor|0.254|0.079|
||Multi-Output Regressor|0.330|0.114|
||K-Nearest Neighbors Regressor|0.258|0.112|
||Gradient Boosting Regressor|0.310|0.111|







**As from above models Multi-Output Regressor have performed well with all the features. The C3D feature with Multi-Output Regressor give best results for both short and long term mmemorability score.**

**We will be using full 6000 records of development datatset to train our model and predict the scores on testing set which has 2000 records.** 

# **# 6. Training Full Dev-Set**

In [0]:
X = captions_bag
y = dataset[['short-term_memorability','long-term_memorability']].values

In [281]:
print(f'X: ({len(X)})')
print(f'y:{y.shape}')

X: (6000)
y:(6000, 2)


In [291]:
final_multirf = MultiOutputRegressor(RandomForestRegressor( random_state=45))
final_multirf.fit(X, y)


MultiOutputRegressor(estimator=RandomForestRegressor(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     criterion='mse',
                                                     max_depth=None,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     max_samples=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
                                                     n_estimators=100,
                                              

**Loading the Testing Data**

In [0]:
#Function to load the captions into dataframes
def load_captions_test(filename):
    video_name_test = []
    captions_test = []
    dataframe_test = pd.DataFrame()
    with open(filename) as file:
        for line in file:
            pair = line.split() #each line in the text file contains to words so, this code will split them into two words
            video_name_test.append(pair[0]) #first word will be assigned as video name
            captions_test.append(pair[1]) #second word will be assigned as caption
        dataframe_test['video']=video_name_test #setting these two as column names of dataframe
        dataframe_test['caption']=captions_test
    return dataframe_test

In [0]:
#load captions
captions_path ='./Test-set/Captions_test/test-set-1_video-captions.txt'
test_captions = load_captions_test(captions_path)

In [294]:
test_captions.head(5)

,video,caption
0,video7494.webm,green-jeep-struggling-to-drive-over-huge-rocks
1,video7495.webm,hiking-woman-tourist-is-walking-forward-in-mou...
2,video7496.webm,close-up-of-african-american-doctors-hands-usi...
3,video7497.webm,slow-motion-of-a-man-using-treadmill-in-the-gy...
4,video7498.webm,slow-motion-of-photographer-in-national-park


In [295]:
#Removing punctuations and stop words from captions
# setup prograss tracker
pbar = pyprind.ProgBar(len(test_captions['caption']), title='Counting word occurrences')
for i, cap in enumerate(test_captions['caption']):
    # replace punctuations with space
    # convert words to lower case 
    text = ''.join([c if c not in punctuation else ' ' for c in cap]).lower()
    #removing stopwords
    rmv_stopwords= ' '.join([word for word in text.split() if word not in stopwords])
    test_captions.loc[i,'caption'] = rmv_stopwords #updating the original captions 
    pbar.update()

Counting word occurrences
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [296]:
test_captions.head()

,video,caption
0,video7494.webm,green jeep struggling drive huge rocks
1,video7495.webm,hiking woman tourist walking forward mountains...
2,video7496.webm,close african american doctors hands using sph...
3,video7497.webm,slow motion man using treadmill gym regular ph...
4,video7498.webm,slow motion photographer national park


In [297]:
#implementing bag of words for the combined captions
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word",max_features=3122) 
test_captions_bag = vectorizer.fit_transform(test_captions.caption).toarray()
type(test_captions_bag)

numpy.ndarray

In [298]:
print(f'Development Vocabulary Size   : {len(captions_bag[0])}')
print(f'Testing Vocabulary Size       : {len(test_captions_bag[0])}')

Development Vocabulary Size   : 3112
Testing Vocabulary Size       : 3112


In [0]:
test_pred = final_multirf.predict(test_captions_bag)


In [0]:
pred = pd.DataFrame()

In [301]:
type(test_pred)

numpy.ndarray

In [0]:
pred['short-term'] = test_pred[:,0]

In [0]:
pred['long-term'] = test_pred[:,1]

In [304]:
pred.head()

,short-term,long-term
0,0.880204,0.749224
1,0.835572,0.794127
2,0.799627,0.773613
3,0.900068,0.862292
4,0.844054,0.723551


In [305]:
pred.describe()

,short-term,long-term
count,2000.000000,2000.000000
mean,0.841895,0.748591
std,0.038783,0.070782
min,0.656527,0.368548
25%,0.820358,0.712203
50%,0.845011,0.753943
75%,0.867141,0.793247
max,0.951780,0.944788


In [0]:
#load the ground truth dataset
csv_path ='./Test-set/Ground-truth_test/ground_truth_template.csv'
GroundTruth_test = pd.read_csv(csv_path)

In [0]:
GroundTruth_test ['short-term_memorability'] =  (1*pred['short-term'])
GroundTruth_test ['long-term_memorability'] =  (1*pred['long-term'])

# **7. Predicting the Final Result**

In [311]:
GroundTruth_test.head(10)

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations
0,7494,0.880204,33,0.749224,12
1,7495,0.835572,34,0.794127,10
2,7496,0.799627,32,0.773613,13
3,7497,0.900068,33,0.862292,10
4,7498,0.844054,33,0.723551,10
5,7499,0.817382,33,0.733659,13
6,7500,0.838796,33,0.689424,10
7,7502,0.849565,43,0.722907,15
8,7503,0.781681,33,0.789779,11
9,7504,0.846573,33,0.667634,15


In [0]:
resultset = GroundTruth_test

In [0]:
resultset.to_csv('/content/gdrive/My Drive/Nivedita_Mahato_19210398_Result.csv',index=None)